In [ ]:
from IPython.display import display, Image, clear_output
import time

#import things for google form
import requests
from bs4 import BeautifulSoup
import json
import time
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok
    
# import things for buttons
# you may need to run
# pip install ipywidgets
# pip install jupyter_ui_poll
# if running on your own computer
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from jupyter_ui_poll import ui_events

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

# the start of the test
print("Welcome to the Approximation Number Sense(ANS) test!")
time.sleep(1.5)
print("In this test, image with blue and yellow dots will be presented.")
time.sleep(1.5)
print("You will be asked to identidy which sides has MORE dots.")
time.sleep(1.5)
clear_output(wait=False)

#consent
print("Before taking the test:")
data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

print(data_consent_info)
result = input("> ") 

if result == "yes": 
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
    time.sleep(3)
    clear_output(wait=False)
else: 
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))

#produce anonymised ID
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifer would be CBTC
"""

print(id_instructions)
user_id = input("> ")

print("User entered id:", user_id)
time.sleep(1.5)
clear_output(wait=False)

#basic information collection
age = input("Please enter your age; \nthis will only be used for educational purpose.")
clear_output(wait=False)
sex = input("Please enter your sex; \nthis will only be used for educational purpose.")
clear_output(wait=False)
print("Before taking the test, if you have any of the following please enter：\n a for coffee or tea or any energy drink; \n b for alcohol or any psychiatric drugs; \n c for none of them.")
drink = input('Have you drink any above?')
clear_output(wait=False)

#info_list
time_taken_list = []
score_each = []
total_score_summary = []
average_time_summary = []

#collect data from the each question

#define the test
def question(image_list, ans_list):
    score = 0
    time_allstart = time.time()
    for i in range(30):
    #showing the image
        display(image_list[i])
        time.sleep(1.5)
        start_time = time.time()
        clear_output(wait=False)

        btn1 = widgets.Button(description="left")
        btn2 = widgets.Button(description="right")
        btn1.on_click(register_event) 
        btn2.on_click(register_event) 
        myhtml1 = HTML("<h1>Which side has more dots?</h1>")
        display(myhtml1)
        myhtml2 = HTML("<h2>You have 3 seconds to answer</h2>")
        display(myhtml2)
        panel = widgets.HBox([btn1, btn2])
        display(panel)
        result = wait_for_event(timeout=3)
        clear_output(wait=False)

        if result['description'] == ans_list[i] :
            print("Correct!")
            score = score + 1
            user_ans[i].append(1)
            score_each.append(score)
        else:
            print("Sorry, you do not get it right in time")
            user_ans[i].append(0)
            score_each.append(score)
            
        end_time = time.time()
        time_taken = end_time - start_time
        time_taken_list.append(time_taken)
        print("time_taken:", time_taken)
        total_score = score /30 * 100
        print(f'score so far = {total_score}/100')
        time.sleep(1.5)
        clear_output(wait=False)
        
    print('Thank you for taking the test \nHere is your result')
    total_score_summary.append(total_score)
    print(f"total score = {total_score}")

    average_time_taken = sum(time_taken_list)/30
    average_time_summary.append(average_time_taken)
    print(f"average time taken: {average_time_taken}")
    time.sleep(5)
    clear_output(wait=False)
    
#store the images
image1 = Image("Slide1.PNG", width = 600)
image2 = Image("Slide2.PNG", width = 600)
image3 = Image("Slide3.PNG", width = 600)
image4 = Image("Slide4.PNG", width = 600)
image5 = Image("Slide5.PNG", width = 600)
image6 = Image("Slide6.PNG", width = 600)
image7 = Image("Slide7.PNG", width = 600)
image8 = Image("Slide8.PNG", width = 600)
image9 = Image("Slide9.PNG", width = 600)
image10 = Image("Slide10.PNG", width = 600)
image11 = Image("Slide11.PNG", width = 600)
image12 = Image("Slide12.PNG", width = 600)
image13 = Image("Slide13.PNG", width = 600)
image14 = Image("Slide14.PNG", width = 600)
image15 = Image("Slide15.PNG", width = 600)
image16 = Image("Slide16.PNG", width = 600)
image17 = Image("Slide17.PNG", width = 600)
image18 = Image("Slide18.PNG", width = 600)
image19 = Image("Slide19.PNG", width = 600)
image20 = Image("Slide20.PNG", width = 600)
image21 = Image("Slide21.PNG", width = 600)
image22 = Image("Slide22.PNG", width = 600)
image23 = Image("Slide23.PNG", width = 600)
image24 = Image("Slide24.PNG", width = 600)
image25 = Image("Slide25.PNG", width = 600)
image26 = Image("Slide26.PNG", width = 600)
image27 = Image("Slide27.PNG", width = 600)
image28 = Image("Slide28.PNG", width = 600)
image29 = Image("Slide29.PNG", width = 600)
image30 = Image("Slide30.PNG", width = 600)
image_list = [image1, image2, image3, image4, image5, image6, image7,image8, image9,image10,image11,image12,image13,image14,image15,image16,image17, image18, image19, image20, image21, image22, image23,image24, image25,image26,image27,image28,image29,image30]
ans_list = ['left','right','left','right','right','left','right','left','right','left','right','right','left','right','right','left','right','left','right','left','left','right','left','left','left','left','left','left','right','left']

#store user ans
ans1 = []
ans2 = []
ans3 = []
ans4 = []
ans5 = []
ans6 = []
ans7 = []
ans8 = []
ans9 = []
ans10 = []
ans11 = []
ans12 = []
ans13 = []
ans14 = []
ans15 = []
ans16 = []
ans17 = []
ans18 = []
ans19 = []
ans20 = []
ans21 = []
ans22 = []
ans23 = []
ans24 = []
ans25 = []
ans26 = []
ans27 = []
ans28 = []
ans29 = []
ans30 = []
user_ans = [ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,ans10,ans11,ans12,ans13,ans14,ans15,ans16,ans17,ans18,ans19,ans20,ans21,ans22,ans23,ans24,ans25,ans26,ans27,ans28,ans29,ans30]


#run function with data:
print("The test is about to start, are you ready for that!")
ready = input("enter ready to start!")
test1 = question(image_list, ans_list)
print(test1)

#send to google form:
results_dict = {
    'user_id': [],
    'age':[],
    'sex':[],
    'drink':[],
    'score': [],
    'average_time_taken': [],
    'score1' :[],
    'score2' :[],
    'score3' :[],
    'score4' :[],
    'score5' :[],
    'score6' :[],
    'score7' :[],
    'score8' :[],
    'score9' :[],
    'score10' :[],
    'score11' :[],
    'score12' :[],
    'score13' :[],
    'score14' :[],
    'score15' :[],
    'score16' :[],
    'score17' :[],
    'score18' :[],
    'score19' :[],
    'score20' :[],
    'score21' :[],
    'score22' :[],
    'score23' :[],
    'score24' :[],
    'score25' :[],
    'score26' :[],
    'score27' :[],
    'score28' :[],
    'score29' :[],
    'score30' :[]}

results_dict['user_id'].append(user_id)
results_dict['age'].append(age)
results_dict['sex'].append(sex)
results_dict['drink'].append(drink)
results_dict['score'].append(total_score_summary)
results_dict['average_time_taken'].append(average_time_summary)
results_dict['score1'].append(ans1)
results_dict['score2'].append(ans2)
results_dict['score3'].append(ans3)
results_dict['score4'].append(ans4)
results_dict['score5'].append(ans5)
results_dict['score6'].append(ans6)
results_dict['score7'].append(ans7)
results_dict['score8'].append(ans8)
results_dict['score9'].append(ans9)
results_dict['score10'].append(ans10)
results_dict['score11'].append(ans11)
results_dict['score12'].append(ans12)
results_dict['score13'].append(ans13)
results_dict['score14'].append(ans14)
results_dict['score15'].append(ans15)
results_dict['score16'].append(ans16)
results_dict['score17'].append(ans17)
results_dict['score18'].append(ans18)
results_dict['score19'].append(ans19)
results_dict['score20'].append(ans20)
results_dict['score21'].append(ans21)
results_dict['score22'].append(ans22)
results_dict['score23'].append(ans23)
results_dict['score24'].append(ans24)
results_dict['score25'].append(ans25)
results_dict['score26'].append(ans26)
results_dict['score27'].append(ans27)
results_dict['score28'].append(ans28)
results_dict['score29'].append(ans29)
results_dict['score30'].append(ans30)

form_url = 'https://docs.google.com/forms/d/e/1FAIpQLScvwiUN5D3HUxma_koghzRp_VFgk-H6gm0g83ikATnzDZLFVQ/viewform?usp=sf_link'
send_to_google_form(results_dict, form_url)

Before taking the test:
DATA CONSENT INFORMATION:

Please read:

we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload.
